**Introduction to Computer Vision and OpenCV:**


	
* Understand the fundamentals of computer vision and its applications.
	
* Learn the basics of the OpenCV library and its core functions.
	
* Setup your development environment with OpenCv
	
* Load and display images using OpenCV

In [1]:
import cv2
image = cv2.imread('/Users/bipulkumar/Desktop/image2.png')
cv2.imshow("Image",image)
cv2.waitKey(0)
cv2.destroyAllWindows()

**Image Processing Techniques using OpenCV:**

* Implement basic image processing techniques such as resizing, cropping, and color manipulation.
	
* Apply various filters and enhancements to images.

In [1]:
import cv2
image = cv2.imread('/Users/bipulkumar/Desktop/image2.png')

resized_image = cv2.resize(image, (250,250))
#image[y_start : y_end , x_start : x_end]
cropped_image = image[150 : 500 , 150 : 400]


#Gaussian Blur
blurred_image = cv2.GaussianBlur(image , (5,5) , 0)
#Canny
edges = cv2.Canny(image , 120 , 220)

#Color Manipulation - Invert color
inverted_image = 255 - image

#Color Manipulation - Gray Scale conversion
gray = cv2.cvtColor(image , cv2.COLOR_BGR2GRAY)


cv2.imshow("Image",image)
cv2.imshow("ReSized Image",resized_image)
cv2.imshow("Cropped",cropped_image)
cv2.imshow("Blurred",blurred_image)
cv2.imshow("Canny",edges)
cv2.imshow("Inverted",inverted_image)
cv2.imshow("Gray Scale",gray)

cv2.waitKey(0)
cv2.destroyAllWindows()

**Object Detection and Tracking:**

* Implement object detection using pre-trained deep learning models (e.g., YOLO or SSD).
	
* Develop real-time object tracking using techniques like the Kalman filter or Mean-Shift algorithm.

In [ ]:
import cv2
import numpy as np
image = cv2.imread('/Users/bipulkumar/Desktop/image2.png')
image = cv2.resize(image, (416,416))

#Load Yolo Model
net = cv2.dnn.readNet('/Users/bipulkumar/Downloads/yolov3.weights',
                      '/Users/bipulkumar/Downloads/yolov3.cfg')


#Perform object detection
scalefactor = 1.0/255.0   #Scale factor for image Normalisation
size = (416,416)      #Size of input image for YoLo model 
mean = (0,0,0)        #Mean values for image Normalisation
swapRB = True         #Swap red and blue channeld for image preprocessing
crop = False          #Do not crop the image during blob creation

blob = cv2.dnn.blobFromImage(image,
                            scalefactor,
                            size,
                            mean,
                            swapRB = swapRB,
                            crop = crop)

blob = cv2.dnn.blobFromImage(image, scalefactor, size, mean, swapRB=swapRB, crop=crop)
net.setInput(blob)
output_layers = net.getUnconnectedOutLayersNames()
outs = net.forward(output_layers)

#Implement object tracking
tracker = cv2.legacy_TrackerMOSSE.create()

#Select ROI (Region of Interest) for tracking
bbox = cv2.selectROI(image, False)

#Initilise the tracker with selected ROI
tracker.init(image,bbox)

#cv2.imshow("output_image",image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
import cv2
import tkinter as tk
from tkinter import filedialog
from PIL import Image, ImageTk

# Load YOLO model and COCO class names
net = cv2.dnn.readNet('yolov3.weights', 'yolov3.cfg')
classes = []
with open('coco.names', 'r') as f:
    classes = f.read().strip().split('\n')

# Create a Tkinter GUI
root = tk.Tk()
root.title('YOLO Object Detection App')

# Function to open an image using a file dialog
def open_image():
    file_path = filedialog.askopenfilename()
    if file_path:
        image = cv2.imread(file_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        detect_objects(image)

# Function to display an OpenCV image in the GUI
def display_image(cv_image):
    pil_image = Image.fromarray(cv_image)
    tk_image = ImageTk.PhotoImage(image=pil_image)
    label.config(image=tk_image)
    label.image = tk_image

# Function to perform object detection
def detect_objects(cv_image):
    height, width = cv_image.shape[:2]

    # Prepare input image for YOLO model
    blob = cv2.dnn.blobFromImage(cv_image, 1/255, (416, 416), swapRB=True, crop=False)
    net.setInput(blob)

    # Get detection results
    outs = net.forward(net.getUnconnectedOutLayersNames())

    # Process detection results
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = scores.argmax()
            confidence = scores[class_id]
            if confidence > 0.7:  # Set a confidence threshold
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)

                x = int(center_x - w / 2)
                y = int(center_y - h / 2)

                # Draw bounding box and class label
                cv2.rectangle(cv_image, (x, y), (x + w, y + h), (0, 255, 0), 1)
                label = f'{classes[class_id]}: {confidence:.2f}'
                cv2.putText(cv_image, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)

    display_image(cv_image)

# Create buttons for opening an image
open_button = tk.Button(root, text="Open Image", command=open_image)

# Create an image display label
label = tk.Label(root)
label.pack()

# Place buttons in the GUI
open_button.pack()

root.mainloop()
